An implementation of encoded $\ket{A^2} = \ket{\sqrt{-iY}} = e^{-i \frac{\pi}{4}Y} \ket{+} = \ket{1}$ preparation in the Steane code using stim.

In particular, $A^2 = XH$, which will be used to implement the gate in stim.

Maybe easiest to just do $Y$ eigenstate, $\ket{+i}$?

In [2]:
from stim import Circuit
from noisy_circuits import noisy_encoded_y,noisy_repetition_encoder,noisy_steane_encoder

The logical $X$ outcome for the repetition-encoded ancilla block indicates which eigenstate the Hadamard test tells us it projected onto.

The logical $Y$ outcome for the Steane-encoded data block will indicate which eigenstate is observed.

If these two observables do not match, then there is a logical error on the output.

Below is a test of the circuit without errors, which should give us no errors and always result in selected trials.

In [5]:
def construct_noisy_circuit(perr:float) ->Circuit:
    noisy_circ = Circuit()

    steaneBlock = [0,1,2,3,4,5,6]
    repnBlock = [7,8,9,10,11,12,13]

    flag = True

    noisy_circ.append("RX",steaneBlock[0])
    noisy_circ.append("S_DAG",steaneBlock[0])
    
    noisy_circ.append("RX",repnBlock[0])

    noisy_circ.append("DEPOLARIZE1",[steaneBlock[0],repnBlock[0]],perr)

    noisy_circ = noisy_steane_encoder(noisy_circ,steaneBlock,perr)
    noisy_circ = noisy_repetition_encoder(noisy_circ,repnBlock,perr,True) # Using flag qubit

    noisy_circ = noisy_encoded_y(noisy_circ,steaneBlock,repnBlock,perr)

    noisy_circ.append("DEPOLARIZE1",repnBlock[:-1],perr) # repn encoder adds flag qubit to repnBlock

    noisy_circ.append("MX",repnBlock[:-1]) # readout transversal logical-X
    noisy_circ.append("MY",steaneBlock) # readout transversal logical-Y
        
    return noisy_circ

In [7]:
c = construct_noisy_circuit(.01)
c.diagram() # The noise in the resulting circuit may be excessive, but better to overestimate it.

q0: -RX-S_DAG-DEPOLARIZE1(0.01)-------------------@-DEPOLARIZE2(0.01)---------------------@-DEPOLARIZE2(0.01)-----------------------X-----------------DEPOLARIZE2(0.01)---------------------X-----------------DEPOLARIZE2(0.01)-----------------------------------------DEPOLARIZE1(0.01)-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------S_DAG-----DEPOLARIZE1(0.01)-------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.01)-------------------------------------------------------------------------------------------------------S-----------------DEPOLARIZE1(0.01)-------------------------------------------------------------------------------------------------------------------------------MY:rec[8]--
                                                   | |                                     | |                                       |                 |                                     |                 |                                                                                                                                                                                                                                                                                                                                                                                                                                                     | |
 q1: ----------RX----------------DEPOLARIZE1(0.01)-|-|-------------------DEPOLARIZE1(0.01)-|-|-----------------@-DEPOLARIZE2(0.01)---@-----------------DEPOLARIZE2(0.01)---------------------|-----------------|-----------------@-DEPOLARIZE2(0.01)---------------------DEPOLARIZE1(0.01)---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------S_DAG-------------DEPOLARIZE1(0.01)-------------------------------------------------------------------------------------------|-|-----------------X-DEPOLARIZE2(0.01)-----------------------------------------------------------------------------------------------------S-----------------DEPOLARIZE1(0.01)-------------------------------------------------------------------------------------------------------------MY:rec[9]--
                                                   | |                                     | |                 | |                                                                           |                 |                 | |                                                                                                                                                                                                                                                                                                                                                                                                                                 | |                 | |
 q2: ----------RX----------------DEPOLARIZE1(0.01)-|-|-------------------DEPOLARIZE1(0.01)-|-|-----------------|-|-------------------DEPOLARIZE1(0.01)-@-----------------DEPOLARIZE2(0.01)---@-----------------DEPOLARIZE2(0.01)-|-|-----------------------------------@-DEPOLARIZE2(0.01)---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------S_DAG-------------DEPOLARIZE1(0.01)-------------------------------------------------------------------------|-|-----------------|-|-----------------X-DEPOLARIZE2(0.01)-------------------------------------------------------------------------------------------------

In [11]:
error_rates = [0,
              1.0*10**(-5), 2.5*10**(-5),5.0*10**(-5),7.5*10**(-5),
              1.0*10**(-4), 2.5*10**(-4),5.0*10**(-4),7.5*10**(-4),
              1.0*10**(-3), 2.5*10**(-3),5.0*10**(-3),7.5*10**(-3),
              1.0*10**(-2)
              ]

results = []

shots = 10_000_000

for perr in error_rates:
    selected = 0
    errors = 0

    noisy_circ = Circuit()
    
    noisy_circ = construct_noisy_circuit(perr)
    sampler = noisy_circ.compile_sampler()
    sample = sampler.sample(shots)

    for rec in sample:
        if not rec[0]:
            xs = rec[1:8]
            ys = rec[8:]

            if not sum(xs)%2: # post-select on expected output
                selected +=1
                errors += not(sum(xs)%2 == sum(ys)%2) # no decoding, only compute transversal values
        
    print(str(perr) + ', ' + str([selected/shots,errors/selected]))
    print()

0, [1.0, 0.0]

1e-05, [0.9992976, 7.185046776856063e-05]

2.5e-05, [0.9982379, 0.00016188525801314496]

5e-05, [0.996449, 0.00032023716216284025]

7.500000000000001e-05, [0.9947454, 0.0004884666971066165]

0.0001, [0.9929677, 0.0006642713554529518]

0.00025, [0.9825647, 0.0016545475325950546]

0.0005, [0.9656574, 0.0033969604540906536]

0.00075, [0.9491449, 0.005271165656582045]

0.001, [0.9333311, 0.0072260530051982625]

0.0025, [0.8458689, 0.02041120083738745]

0.005, [0.7287871, 0.047832624918854906]

0.0075, [0.6393906, 0.08121561374221016]

0.01, [0.5707841, 0.117900796465774]

